## 隐形马尔可夫模型（HMM）

隐形马尔可夫模型，英文是 Hidden Markov Models，所以以下就简称 HMM。既是马尔可夫模型，就一定存在马尔可夫链，马尔可夫链服从马尔可夫性质：即无记忆性。也就是说，这一时刻的状态，受且只受前一时刻的影响，而不受更往前时刻的状态的影响。


**HMM的5要素**  
- $N$，隐藏的状态数量
- $M$，观测集值的数量
- $A = [a_{ij}]_{N \times N} $，状态转移概率矩阵
- $B = [b_{ij}]_{N \times M} $，观测值概率矩阵
- $\pi = [\pi_{i}]_{N} $ ，初始状态矩阵

![HMM](../../image/hmm.jpg)

**天气转移概率图**
![天气转移概率图](../../image/hmm_weather.jpg)


**天气活动转移概率图**
![天气活动转移概率图](../../image/hmm_weather_activity.jpg)

**三个问题**
- 问题1，已知整个模型，我女朋友告诉我，连续三天，她下班后做的事情分别是：散步，购物，收拾。那么，根据模型，计算产生这些行为的概率是多少。   
 已知模型参数，计算某一给定可观察状态序列的概率
- 问题2，同样知晓这个模型，同样是这三件事，我女朋友要我猜，这三天她下班后北京的天气是怎么样的。这三天怎么样的天气才最有可能让她做这样的事情。  
 根据可观察状态的序列找到一个最可能的隐藏状态序列
- 问题3，最复杂的，我女朋友只告诉我这三天她分别做了这三件事，而其他什么信息我都没有。她要我建立一个模型，晴雨转换概率，第一天天气情况的概率分布，根据天气情况她选择做某事的概率分布。（惨绝人寰）(不知道咋做)  
  根据观察到的序列集来找到一个最有可能的 HMM。


**参考**

- [如何用简单易懂的例子解释隐马尔可夫模型？](https://www.zhihu.com/question/20962240)
- [HMM的介绍及实现](https://www.cnblogs.com/luxiaoxun/archive/2013/05/12/3074510.html)
- [一文搞懂HMM（隐马尔可夫模型）](http://www.cnblogs.com/skyme/p/4651331.html)

In [2]:
import numpy as np
import pandas as pd

# print(inspect.getargspec(np.sum))

weather = np.array([[0.7, 0.3],[0.4, 0.6]])
weather_activity = np.array([[0.1, 0.4, 0.5],[0.6, 0.3, 0.1]])

pd_weather = pd.DataFrame(weather, index=['rainy', 'sunny'], columns=['rainy', 'sunny'])
pd_weather_activity = pd.DataFrame(weather_activity, index=['rainy', 'sunny'], columns=['walk', 'shop', 'clean'])

print("\n当天天气和第二天天气的矩阵\n", pd_weather)
print("\n天气和当天活动的矩阵\n", pd_weather_activity)

weather_activity2 = weather.dot(weather_activity)
pd_weather_activity2 = pd.DataFrame(weather_activity2, index=['rainy', 'sunny'], columns=['walk', 'shop', 'clean'])
print("\n当天天气和第二天活动的矩阵\n", pd_weather_activity2)

weather2 = weather.dot(weather)
pd_weather2 =  pd.DataFrame(weather2, index=['rainy', 'sunny'], columns=['rainy', 'sunny'])
print("\n当天天气和第三天天气的矩阵\n", pd_weather2)

print("-----------------------------------------------------------------------")
print("问题1，已知整个模型，我女朋友告诉我，连续三天，她下班后做的事情分别是：散步，购物，收拾。那么，根据模型，计算产生这些行为的概率是多少")
walk_day1 = np.array([0.6, 0.4])*weather_activity[:,0]
pd_walk_day1 = pd.DataFrame(walk_day1.reshape((1,2)), index=['walk'], columns=['rainy', 'sunny'])
print("\n第一天散步\n", pd_walk_day1)
print( np.sum(walk_day1))

walk_day2 = walk_day1.dot(weather)*weather_activity[:,1]
pd_walk_day2 = pd.DataFrame(walk_day2.reshape((1,2)), index=['shop'], columns=['rainy', 'sunny'])
print("\n第二天购物|第一天散步\n", pd_walk_day2)
print(np.sum(walk_day2))

walk_day3 = walk_day2.dot(weather)*weather_activity[:,2]
pd_walk_day3 = pd.DataFrame(walk_day3.reshape((1,2)), index=['shop'], columns=['rainy', 'sunny'])
print("\n第三天收拾|第二天购物|第一天散步\n", pd_walk_day3)

print( np.sum(walk_day3))

print("-----------------------------------------------------------------------")
print("问题2，同样知晓这个模型，同样是这三件事，我女朋友要我猜，这三天她下班后北京的天气是怎么样的。这三天怎么样的天气才最有可能让她做这样的事情。")
walk_day1 = np.array([0.6, 0.4])*(weather_activity[:,0])
pd_walk_day1 = pd.DataFrame(walk_day1.reshape((1,2)), index=['walk'], columns=['rainy', 'sunny'])
print("\n第一天散步\n", pd_walk_day1)
print( np.sum(walk_day1))

walk_day2 = walk_day1.reshape((2, 1))*(weather)*weather_activity[:,1].reshape((1, 2))
pd_walk_day2 = pd.DataFrame(walk_day2, index=['rainy', 'sunny'], columns=['rainy', 'sunny'])
print("\n第二天购物|第一天散步\n", pd_walk_day2)
print( np.sum(walk_day2))

walk_day3 = np.array([day2.reshape((2, 1))*(weather)*weather_activity[:,2].reshape((1, 2))  for day2 in walk_day2])
# pd_walk_day3 = pd.DataFrame(walk_day3.reshape((1,2)), index=['shop'], columns=['rainy', 'sunny'])

print("\n第三天收拾|第二天购物|第一天散步\n")
print("rainy\n", pd.DataFrame(walk_day3[0], index=['rainy', 'sunny'], columns=['rainy', 'sunny']))
print("sunny\n", pd.DataFrame(walk_day3[1], index=['rainy', 'sunny'], columns=['rainy', 'sunny']))
   
print(np.sum(walk_day3))

index = np.unravel_index(walk_day3.argmax(), walk_day3.shape)
print('最有可能天气：', ["rainny" if i==0 else "sunny" for i in index])

# 对于问题一，二的求解，我们可以总结成一个公式

# def next(p, A, B, B_index)：
#      pass



当天天气和第二天天气的矩阵
        rainy  sunny
rainy    0.7    0.3
sunny    0.4    0.6

天气和当天活动的矩阵
        walk  shop  clean
rainy   0.1   0.4    0.5
sunny   0.6   0.3    0.1

当天天气和第二天活动的矩阵
        walk  shop  clean
rainy  0.25  0.37   0.38
sunny  0.40  0.34   0.26

当天天气和第三天天气的矩阵
        rainy  sunny
rainy   0.61   0.39
sunny   0.52   0.48
-----------------------------------------------------------------------
问题1，已知整个模型，我女朋友告诉我，连续三天，她下班后做的事情分别是：散步，购物，收拾。那么，根据模型，计算产生这些行为的概率是多少

第一天散步
       rainy  sunny
walk   0.06   0.24
0.3

第二天购物|第一天散步
        rainy   sunny
shop  0.0552  0.0486
0.1038

第三天收拾|第二天购物|第一天散步
         rainy     sunny
shop  0.02904  0.004572
0.033612
-----------------------------------------------------------------------
问题2，同样知晓这个模型，同样是这三件事，我女朋友要我猜，这三天她下班后北京的天气是怎么样的。这三天怎么样的天气才最有可能让她做这样的事情。

第一天散步
       rainy  sunny
walk   0.06   0.24
0.3

第二天购物|第一天散步
         rainy   sunny
rainy  0.0168  0.0054
sunny  0.0384  0.0432
0.1038

第三天收拾|第二天购物|第一天散步

rainy
          rainy     sunny
rainy  